In [1]:
!pip install -q transformers gradio PyPDF2 sentencepiece

print("Libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.6 MB/s eta 0:00:00
Libraries installed successfully!


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# I use 'flan-t5-large' because it's great at following instructions.
checkpoint = "google/flan-t5-large"

print(f"Downloading and loading {checkpoint}... (This might take 1-2 minutes)")

# The tokenizer acts like a translator: converts words -> numbers for the AI.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Model loaded successfully on the GPU!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded successfully on the GPU!


In [5]:
import textwrap
from PyPDF2 import PdfReader

# Read the PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            full_text += text + "\n"
    return full_text

# The Core AI Processing
def simplify_text(text):
    # Smaller chunks (1500 chars) to avoid the "527 > 512" error
    chunks = textwrap.wrap(text, 1500)

    simplified_output = []
    print(f"Processing {len(chunks)} chunks of text...")

    for i, chunk in enumerate(chunks):
        # Slightly more mature prompt to stop hardware hallucinations
        prompt = "Summarize the key points of this text in clear, simple English: " + chunk

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

        # Added parameters to stop looping (repetition_penalty)
        outputs = model.generate(
            input_ids,
            max_new_tokens=200,
            min_length=50,          # Force it to write at least a decent sentence
            repetition_penalty=2.0, # STRICT penalty for repeating words
            no_repeat_ngram_size=3  # Cannot repeat any 3-word phrase
        )

        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        simplified_output.append(decoded_output)

        print(f"Finished chunk {i+1}/{len(chunks)}")

    # Join with double newlines for better reading
    return "\n\n".join(simplified_output)

print("Logic functions UPDATED successfully!")

Logic functions UPDATED successfully!


In [8]:
!pip install -q nest_asyncio

import nest_asyncio
nest_asyncio.apply()

import gradio as gr

# THE BRIDGE FUNCTION
def process_pipeline(pdf_file):
    if pdf_file is None:
        return "Please upload a PDF file first!"

    print(f"File received: {pdf_file.name}")

    # Extract text
    try:
        raw_text = extract_text_from_pdf(pdf_file.name)
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

    print("Text extracted, sending to AI...")

    # Simplify
    try:
        simple_version = simplify_text(raw_text)
    except Exception as e:
        return f"Error during simplification: {str(e)}"

    return simple_version

# THE UI SETUP
demo = gr.Interface(
    fn=process_pipeline,
    inputs=gr.File(label="Upload Research Paper (PDF)"),
    outputs=gr.Textbox(label="Simplified Version", lines=15),
    title="The Jargon Buster 🤖",
    description="Upload a complex PDF. This AI will summarize key points in simple English."
)

print("Starting UI...")
demo.launch(share=True, debug=False)

Starting UI...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e39d0bf6565679347.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
